In [9]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. 데이터 불러오기
csv_file = '../../dataset/inference/travel/241004_노선모두_8.28~9.11_평일_속도X/0926_평일_속도X_모든노선_추론결과.csv'
df = pd.read_csv(csv_file)

# 2. 실제 이벤트 시간 데이터를 분 단위로 변환
df['TRAVEL_TIME_EVENT_MIN'] = df['TRAVEL_TIME_EVENT'] / 60  # 실제 이벤트 소통 시간 분 단위 변환
df['SERVICE_TIME_EVENT_MIN'] = df['SERVICE_TIME_EVENT'] / 60  # 실제 이벤트 정차 시간 분 단위 변환

# 3. 데이터 필터링: 소통시간과 정차시간 관련 데이터에서 AI, BIS, 실제 이벤트 값이 모두 존재하는 행만 남기기
# 소통시간 관련 데이터 필터링
df_filtered_travel = df[['TRAVEL_TIME_EVENT_MIN', 'DIFF_EVENT_AI', 'DIFF_EVENT_BIS']].dropna().copy()
# df_filtered_travel = df_filtered_travel[df_filtered_travel['TRAVEL_TIME_EVENT_MIN'] > 1]
df_filtered_travel = df_filtered_travel[df_filtered_travel['TRAVEL_TIME_EVENT_MIN'] >= 0]

# 정차시간 관련 데이터 필터링
df_filtered_service = df[['SERVICE_TIME_EVENT_MIN', 'DIFF_EVENT_SERVICE_AI', 'DIFF_EVENT_SERVICE_BIS']].dropna().copy()
# df_filtered_service = df_filtered_service[df_filtered_service['SERVICE_TIME_EVENT_MIN'] > 1]
df_filtered_service = df_filtered_service[df_filtered_service['SERVICE_TIME_EVENT_MIN'] >= 0]
df_filtered_service = df_filtered_service[df_filtered_service['SERVICE_TIME_EVENT_MIN'] <= 5]

# 4. 구간화: 1분 단위로 구간을 나눕니다.
# TRAVEL_TIME_EVENT_MIN 구간화
max_travel = df_filtered_travel['TRAVEL_TIME_EVENT_MIN'].max()
# travel_bins = list(range(1, int(max_travel) + 2))  # 1부터 최대 시간 +1까지
# travel_labels = [f"{i}-{i+1}분" for i in range(1, int(max_travel) + 1)]
travel_bins = list(range(0, int(max_travel) + 2))  # 0부터 최대 시간 +1까지
travel_labels = [f"{i}-{i+1}분" for i in range(0, int(max_travel) + 1)]
df_filtered_travel['TRAVEL_BIN'] = pd.cut(df_filtered_travel['TRAVEL_TIME_EVENT_MIN'],
                                         bins=travel_bins,
                                         labels=travel_labels,
                                         right=False)

# SERVICE_TIME_EVENT_MIN 구간화
max_service = df_filtered_service['SERVICE_TIME_EVENT_MIN'].max()
# service_bins = list(range(1, int(max_service) + 2))
# service_labels = [f"{i}-{i+1}분" for i in range(1, int(max_service) + 1)]
service_bins = list(range(0, int(max_service) + 2))
service_labels = [f"{i}-{i+1}분" for i in range(0, int(max_service) + 1)]
df_filtered_service['SERVICE_BIN'] = pd.cut(df_filtered_service['SERVICE_TIME_EVENT_MIN'],
                                           bins=service_bins,
                                           labels=service_labels,
                                           right=False)

# 5. 구간별 평균 오차 계산
# TRAVEL_TIME_EVENT_MIN 구간별
travel_group = df_filtered_travel.groupby('TRAVEL_BIN').agg(
    DIFF_EVENT_AI_MEAN=('DIFF_EVENT_AI', 'mean'),   # 실제 이벤트 - AI 소통시간 예측
    DIFF_EVENT_BIS_MEAN=('DIFF_EVENT_BIS', 'mean'),  # 실제 이벤트 - BIS 소통시간 예측
    COUNT=('TRAVEL_TIME_EVENT_MIN', 'size')          # 구간별 데이터 개수
).reset_index()

# SERVICE_TIME_EVENT 구간별
service_group = df_filtered_service.groupby('SERVICE_BIN').agg(
    DIFF_EVENT_SERVICE_AI_MEAN=('DIFF_EVENT_SERVICE_AI', 'mean'),   # 실제 이벤트 - AI 정차시간 예측
    DIFF_EVENT_SERVICE_BIS_MEAN=('DIFF_EVENT_SERVICE_BIS', 'mean'),  # 실제 이벤트 - BIS 정차시간 예측
    COUNT=('SERVICE_TIME_EVENT_MIN', 'size')                        # 구간별 데이터 개수
).reset_index()

# 6. 전체 평균 오차 및 향상 비율 계산
# TRAVEL_TIME_EVENT 전체 평균 오차 계산
mean_diff_event_ai_travel = df_filtered_travel['DIFF_EVENT_AI'].mean()
mean_diff_event_bis_travel = df_filtered_travel['DIFF_EVENT_BIS'].mean()

# SERVICE_TIME_EVENT 전체 평균 오차 계산
mean_diff_event_ai_service = df_filtered_service['DIFF_EVENT_SERVICE_AI'].mean()
mean_diff_event_bis_service = df_filtered_service['DIFF_EVENT_SERVICE_BIS'].mean()

# 소통시간 정확도 향상 비율 계산 (%)
improvement_travel = ((mean_diff_event_bis_travel - mean_diff_event_ai_travel) / mean_diff_event_bis_travel) * 100 if mean_diff_event_bis_travel != 0 else 0

# 정차시간 정확도 향상 비율 계산 (%)
improvement_service = ((mean_diff_event_bis_service - mean_diff_event_ai_service) / mean_diff_event_bis_service) * 100 if mean_diff_event_bis_service != 0 else 0

# 7. 소수점 없애기 위해 숫자를 정수로 변환하기 전에 NaN 값을 처리
travel_group['DIFF_EVENT_AI_MEAN'] = travel_group['DIFF_EVENT_AI_MEAN'].fillna(0).round(0).astype(int)
travel_group['DIFF_EVENT_BIS_MEAN'] = travel_group['DIFF_EVENT_BIS_MEAN'].fillna(0).round(0).astype(int)
service_group['DIFF_EVENT_SERVICE_AI_MEAN'] = service_group['DIFF_EVENT_SERVICE_AI_MEAN'].fillna(0).round(0).astype(int)
service_group['DIFF_EVENT_SERVICE_BIS_MEAN'] = service_group['DIFF_EVENT_SERVICE_BIS_MEAN'].fillna(0).round(0).astype(int)

# 8. Plotly 시각화
# 서브플롯 생성: 2행 1열
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("소통시간 이벤트 오차", "정차시간 이벤트 오차"),
                    shared_xaxes=False)

# 첫 번째 서브플롯: Travel Time
fig.add_trace(
    go.Bar(
        x=travel_group['TRAVEL_BIN'],
        y=travel_group['DIFF_EVENT_AI_MEAN'],
        name='AI 모델',
        marker_color='indianred',
        text=travel_group['DIFF_EVENT_AI_MEAN'],  # 평균 오차 값을 텍스트로 표시
        textposition='outside'  # 텍스트를 막대 위에 표시
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=travel_group['TRAVEL_BIN'],
        y=travel_group['DIFF_EVENT_BIS_MEAN'],
        name='현 BIS',
        marker_color='lightsalmon',
        text=travel_group['DIFF_EVENT_BIS_MEAN'],  # 평균 오차 값을 텍스트로 표시
        textposition='outside'  # 텍스트를 막대 위에 표시
    ),
    row=1, col=1
)

# 두 번째 서브플롯: Service Time
fig.add_trace(
    go.Bar(
        x=service_group['SERVICE_BIN'],
        y=service_group['DIFF_EVENT_SERVICE_AI_MEAN'],
        name='AI 모델',
        marker_color='mediumseagreen',
        text=service_group['DIFF_EVENT_SERVICE_AI_MEAN'],  # 평균 오차 값을 텍스트로 표시
        textposition='outside'  # 텍스트를 막대 위에 표시
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(
        x=service_group['SERVICE_BIN'],
        y=service_group['DIFF_EVENT_SERVICE_BIS_MEAN'],
        name='현 BIS',
        marker_color='lightgreen',
        text=service_group['DIFF_EVENT_SERVICE_BIS_MEAN'],  # 평균 오차 값을 텍스트로 표시
        textposition='outside'  # 텍스트를 막대 위에 표시
    ),
    row=2, col=1
)

# 텍스트 주석 추가
fig.add_annotation(
    text=f"AI 모델 예측 평균 오차: {mean_diff_event_ai_travel:.1f}초<br>현 BIS 예측 평균 오차: {mean_diff_event_bis_travel:.1f}초<br>현 BIS 대비 AI 모델 정확도 향상: {improvement_travel:.1f}%",
    xref="x domain",
    yref="y domain",
    x=0.5,
    y=0.95,
    showarrow=False,
    font=dict(size=12, color="indianred"),
    align="center",
    row=1,
    col=1
)

fig.add_annotation(
    text=f"AI 모델 예측 평균 오차: {mean_diff_event_ai_service:.1f}초<br>현 BIS 예측 평균 오차: {mean_diff_event_bis_service:.1f}초<br>현 BIS 대비 AI 모델 정확도 향상: {improvement_service:.1f}%",
    xref="x domain",
    yref="y domain",
    x=0.5,
    y=0.95,
    showarrow=False,
    font=dict(size=12, color="mediumseagreen"),
    align="center",
    row=2,
    col=1
)

# 첫 번째 서브플롯의 X축에 데이터 개수 표시 (소통시간)
for i, (bin_label, count) in enumerate(zip(travel_group['TRAVEL_BIN'], travel_group['COUNT'])):
    fig.add_annotation(
        text=f"{count}개",  # 데이터 개수 표시
        x=bin_label,
        y=-5,  # X축 바로 아래에 표시
        xref="x",  # X축 기준으로 위치 설정
        yref="paper",  # Y축은 전체 그래프 영역 기준으로 설정
        showarrow=False,
        font=dict(size=12),
        row=1,  # 첫 번째 서브플롯
        col=1
    )

# 두 번째 서브플롯의 X축에 데이터 개수 표시 (정차시간)
for i, (bin_label, count) in enumerate(zip(service_group['SERVICE_BIN'], service_group['COUNT'])):
    fig.add_annotation(
        text=f"{count}개",  # 데이터 개수 표시
        x=bin_label,
        y=-5,  # X축 바로 아래에 표시
        xref="x",  # X축 기준으로 위치 설정
        yref="paper",  # Y축은 전체 그래프 영역 기준으로 설정
        showarrow=False,
        font=dict(size=12),
        row=2,  # 두 번째 서브플롯
        col=1
    )


# 레이아웃 업데이트 (수정)
fig.update_layout(
    title_text="AI 모델과 BIS 시스템의 Travel 및 Service Time 예측 오차 비교",
    barmode='group',
    height=1000,
    margin=dict(t=150)  # 상단 마진을 늘려 주석 공간 확보
)

# 소통시간 (Travel Time) 서브플롯의 X축 레이블 스타일 변경
fig.update_xaxes(
    title_text="실제 이벤트 소통시간 (분)", 
    row=1, 
    col=1,
    tickfont=dict(size=14, color='white')  # 글자 크기와 색상 설정
)
fig.update_yaxes(title_text="평균 오차 (초)", row=1, col=1)

# 정차시간 (Service Time) 서브플롯의 X축 레이블 스타일 변경
fig.update_xaxes(
    title_text="실제 이벤트 정차시간 (분)", 
    row=2, 
    col=1,
    tickfont=dict(size=14, color='white')  # 글자 크기와 색상 설정
)
fig.update_yaxes(title_text="평균 오차 (초)", row=2, col=1)

# 범례 위치 조정
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

# 그래프 표시
fig.show()


C:\Users\seung\AppData\Local\Temp\ipykernel_5180\349242266.py:50: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\seung\AppData\Local\Temp\ipykernel_5180\349242266.py:57: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# '12-13분'에 해당하는 실제 CSV 데이터의 행들을 필터링
travel_filtered_rows = df_filtered_travel[df_filtered_travel['TRAVEL_BIN'] == '5-6분']

# 필터된 데이터 출력
print(travel_filtered_rows)

# 소통 + 정차 합친 모델


In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. 데이터 불러오기
csv_file = '../../dataset/inference/route/241007_모든노선_8.1~8.14_평일_특성추가/0821_소통+정차_특성추가_추론결과.csv'
df = pd.read_csv(csv_file)

# 2. 실제 이벤트 시간 데이터를 분 단위로 변환
df['TRAVEL_TIME_EVENT_MIN'] = df['TRAVEL_TIME_EVENT'] / 60  # 실제 이벤트 소통 시간 분 단위 변환

# 3. 데이터 필터링: 소통시간 관련 데이터에서 AI, BIS, 실제 이벤트 값이 모두 존재하는 행만 남기기
df_filtered_travel = df[['TRAVEL_TIME_EVENT_MIN', 'DIFF_EVENT_AI', 'DIFF_EVENT_BIS']].dropna().copy()

# 4. 구간화: 1분 단위로 구간을 나눕니다 (0-1분 포함)
max_travel = df_filtered_travel['TRAVEL_TIME_EVENT_MIN'].max()
travel_bins = list(range(0, int(max_travel) + 2))  # 0부터 최대 시간 +1까지
travel_labels = [f"{i}-{i+1}분" for i in range(0, int(max_travel) + 1)]
df_filtered_travel['TRAVEL_BIN'] = pd.cut(df_filtered_travel['TRAVEL_TIME_EVENT_MIN'],
                                         bins=travel_bins,
                                         labels=travel_labels,
                                         right=False)

# 5. 구간별 평균 오차 및 데이터 개수 계산
travel_group = df_filtered_travel.groupby('TRAVEL_BIN').agg(
    DIFF_EVENT_AI_MEAN=('DIFF_EVENT_AI', 'mean'),    # 실제 이벤트 - AI 소통시간 예측 오차
    DIFF_EVENT_BIS_MEAN=('DIFF_EVENT_BIS', 'mean'),  # 실제 이벤트 - BIS 소통시간 예측 오차
    COUNT=('TRAVEL_TIME_EVENT_MIN', 'size')          # 구간별 데이터 개수
).reset_index()

# 6. 전체 평균 오차 및 향상 비율 계산
mean_diff_event_ai_travel = df_filtered_travel['DIFF_EVENT_AI'].mean()
mean_diff_event_bis_travel = df_filtered_travel['DIFF_EVENT_BIS'].mean()

# 소통시간 정확도 향상 비율 계산 (%)
improvement_travel = ((mean_diff_event_bis_travel - mean_diff_event_ai_travel) / mean_diff_event_bis_travel) * 100 if mean_diff_event_bis_travel != 0 else 0

# 7. 소수점 없애기 위해 숫자를 정수로 변환하기 전에 NaN 값을 처리
travel_group['DIFF_EVENT_AI_MEAN'] = travel_group['DIFF_EVENT_AI_MEAN'].fillna(0).round(0).astype(int)
travel_group['DIFF_EVENT_BIS_MEAN'] = travel_group['DIFF_EVENT_BIS_MEAN'].fillna(0).round(0).astype(int)
travel_group['COUNT'] = travel_group['COUNT'].fillna(0).astype(int)

# 8. 구간 레이블에 데이터 개수 추가
travel_group['TRAVEL_BIN_LABEL'] = travel_group.apply(lambda row: f"{row['TRAVEL_BIN']}<br>{row['COUNT']}개", axis=1)

# 9. Plotly 시각화
# 서브플롯 생성: 1행 1열 (소통시간만)
fig = make_subplots(rows=1, cols=1,
                    subplot_titles=("소통시간 이벤트 오차"),
                    shared_xaxes=False)

# 소통시간 이벤트 오차 시각화
fig.add_trace(
    go.Bar(
        x=travel_group['TRAVEL_BIN_LABEL'],
        y=travel_group['DIFF_EVENT_AI_MEAN'],
        name='AI 모델',
        marker_color='indianred',
        text=[f"{mean}초<br>({count}개)" for mean, count in zip(travel_group['DIFF_EVENT_AI_MEAN'], travel_group['COUNT'])],  # 평균 오차와 데이터 개수 결합
        textposition='outside'  # 막대 위에 텍스트 위치
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=travel_group['TRAVEL_BIN_LABEL'],
        y=travel_group['DIFF_EVENT_BIS_MEAN'],
        name='현 BIS',
        marker_color='lightsalmon',
        text=[f"{mean}초<br>({count}개)" for mean, count in zip(travel_group['DIFF_EVENT_BIS_MEAN'], travel_group['COUNT'])],  # 평균 오차와 데이터 개수 결합
        textposition='outside'  # 막대 위에 텍스트 위치
    ),
    row=1, col=1
)

# 텍스트 주석 추가 (전체 평균 오차 및 향상 비율)
fig.add_annotation(
    text=f"AI 모델 예측 평균 오차: {mean_diff_event_ai_travel:.1f}초<br>현 BIS 예측 평균 오차: {mean_diff_event_bis_travel:.1f}초<br>현 BIS 대비 AI 모델 정확도 향상: {improvement_travel:.1f}%",
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.05,
    showarrow=False,
    font=dict(size=14, color="indianred"),
    align="center"
)

# 레이아웃 업데이트
fig.update_layout(
    title_text="AI 모델과 BIS 시스템의 소통시간 예측 오차 비교",
    barmode='group',
    height=600,
    margin=dict(t=150)  # 상단 마진을 늘려 주석 공간 확보
)

# X축 레이블 스타일 변경
fig.update_xaxes(
    title_text="실제 이벤트 소통시간 (분)",
    tickfont=dict(size=14, color='white'),  # 글자 크기와 색상 설정
    row=1,
    col=1
)

# Y축 레이블 설정
fig.update_yaxes(
    title_text="평균 오차 (초)",
    tickfont=dict(size=14, color='white'),
    row=1,
    col=1
)

# 범례 위치 조정
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

# 그래프 표시
fig.show()


C:\Users\seung\AppData\Local\Temp\ipykernel_9500\2767460145.py:25: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [4]:
# '12-13분'에 해당하는 실제 CSV 데이터의 행들을 필터링
travel_filtered_rows = df_filtered_travel[df_filtered_travel['TRAVEL_BIN'] == '6-7분']

# 필터된 데이터 출력
print(travel_filtered_rows)

        TRAVEL_TIME_EVENT_MIN  DIFF_EVENT_AI  DIFF_EVENT_BIS TRAVEL_BIN
25694                6.700000          294.0          1153.0       6-7분
285422               6.741667          213.5           119.5       6-7분
